In [ ]:
import pandas as pd
import numpy as np
import json
import os
import joblib
import torch
from tqdm.notebook import tqdm
from transformers import BertModel
from transformers import BertTokenizer


In [ ]:
df=# load your data

In [ ]:
# Load fine-tuned rubert-tiny2 model to obtain new embeddings:
rc_path='../models/fine-tuned_rubert-tiny2/v2_8_1536_1.2/'

model = BertModel.from_pretrained(rc_path)
tokenizer = BertTokenizer.from_pretrained(rc_path)
model_emb = model.base_model

# Get embeddings for new data based on fine-tuned model

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [ ]:
#%%time
from tqdm.notebook import tqdm
emb_list = list()
it = 0
for s in tqdm(df['text']):
    emb_list.append(embed_bert_cls(s, model_emb, tokenizer))

In [ ]:
# SAVE embeddings
file_name=#file to save emb
np.save(file_name, emb_list)

In [ ]:
# Load SVM model to predict if the posts are value-expressive (1/0):
filename_SVM='../models/fine-tuned_rubert-tiny2/SVC_for_binary_classification_pre-trained_rubert-tiny2_based.pkl'   
loaded_model = joblib.load(filename_SVM)

In [ ]:
#predict
y_predict=loaded_model.predict_proba(emb_list)
df['if_value']=(y_predict[:,1] >= 0.42).astype(int)

In [ ]:
# for further analysis we need posts that are value-expressive and/or politically-oriented
df['if_political_OR_if_value'] = np.where((df['If_political'] == 1)|(df['if_value'] == 1),1, 0)
df['if_political_OR_if_value'].value_counts()

In [ ]:
#save df